# Quantizing RNN Models

In this example, we show how to quantize recurrent models.  
Using a pretrained model `model.RNNModel`, we convert the built-in pytorch implementation of LSTM to our own, modular implementation.  
The pretrained model was generated with:  
```time python3 main.py --cuda --emsize 1500 --nhid 1500 --dropout 0.65 --tied --wd=1e-6```  
The reason we replace the LSTM that is because the inner operations in the pytorch implementation are not accessible to us, but we still want to quantize these operations. <br />
Afterwards we can try different techniques to quantize the whole model.  

_NOTE_: We use `tqdm` to plot progress bars, since it's not in `requirements.txt` you should install it using 
`pip install tqdm`.

In [1]:
from model import DistillerRNNModel, RNNModel
from data import Corpus
import torch
from torch import nn
import distiller
from distiller.modules import DistillerLSTM as LSTM
from tqdm import tqdm # for pretty progress bar
import numpy as np
from copy import deepcopy

### Preprocess the data

In [2]:
corpus = Corpus('./data/wikitext-2/')

In [3]:
def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)
device = 'cuda:0'
batch_size = 20
eval_batch_size = 10
train_data = batchify(corpus.train, batch_size)
val_data = batchify(corpus.valid, eval_batch_size)
test_data = batchify(corpus.test, eval_batch_size)

### Loading the model and converting to our own implementation

In [4]:
rnn_model = torch.load('./checkpoint.pth.tar.best')
rnn_model = rnn_model.to(device)
rnn_model

RNNModel(
  (drop): Dropout(p=0.65)
  (encoder): Embedding(33278, 1500)
  (rnn): LSTM(1500, 1500, num_layers=2, dropout=0.65)
  (decoder): Linear(in_features=1500, out_features=33278, bias=True)
)

Here we convert the pytorch LSTM implementation to our own, by calling `LSTM.from_pytorch_impl`:

In [5]:
def manual_model(pytorch_model_: RNNModel):
    nlayers, ninp, nhid, ntoken, tie_weights = \
        pytorch_model_.nlayers, \
        pytorch_model_.ninp, \
        pytorch_model_.nhid, \
        pytorch_model_.ntoken, \
        pytorch_model_.tie_weights

    model = DistillerRNNModel(nlayers=nlayers, ninp=ninp, nhid=nhid, ntoken=ntoken, tie_weights=tie_weights).to(device)
    model.eval()
    model.encoder.weight = nn.Parameter(pytorch_model_.encoder.weight.clone().detach())
    model.decoder.weight = nn.Parameter(pytorch_model_.decoder.weight.clone().detach())
    model.decoder.bias = nn.Parameter(pytorch_model_.decoder.bias.clone().detach())
    model.rnn = LSTM.from_pytorch_impl(pytorch_model_.rnn)

    return model

man_model = manual_model(rnn_model)
torch.save(man_model, 'manual.checkpoint.pth.tar')
man_model

DistillerRNNModel(
  (encoder): Embedding(33278, 1500)
  (rnn): DistillerLSTM(1500, 1500, num_layers=2, dropout=0.65, bidirectional=False)
  (decoder): Linear(in_features=1500, out_features=33278, bias=True)
)

### Batching the data for evaluation

In [6]:
sequence_len = 35
def get_batch(source, i):
    seq_len = min(sequence_len, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

hidden = rnn_model.init_hidden(eval_batch_size)
data, targets = get_batch(test_data, 0)

### Check that the convertion has succeeded

In [7]:
rnn_model.eval()
man_model.eval()
y_t, h_t = rnn_model(data, hidden)
y_p, h_p = man_model(data, hidden)

print("Max error in y: %f" % (y_t-y_p).abs().max().item())

Max error in y: 0.000007


### Defining the evaluation

In [8]:
criterion = nn.CrossEntropyLoss()
def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)
    

def evaluate(model, data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0.
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(eval_batch_size)
    with torch.no_grad():
        with tqdm(range(0, data_source.size(0), sequence_len)) as t:
            # The line below was fixed as per: https://github.com/pytorch/examples/issues/214
            for i in t:
                data, targets = get_batch(data_source, i)
                output, hidden = model(data, hidden)
                output_flat = output.view(-1, ntokens)
                total_loss += len(data) * criterion(output_flat, targets).item()
                hidden = repackage_hidden(hidden)
                avg_loss = total_loss / (i + 1)
                t.set_postfix((('val_loss', avg_loss), ('ppl', np.exp(avg_loss))))
    return total_loss / len(data_source)

# Quantizing the Model

## Collect activation statistics

The model uses activation statistics to determine how big the quantization range is. The bigger the range - the larger the round off error after quantization which leads to accuracy drop.  
Our goal is to minimize the range s.t. it contains the absolute most of our data.  
After that, we divide the range into chunks of equal size, according to the number of bits, and transform the data according to this scale factor.  
Read more on scale factor calculation [in our docs](https://nervanasystems.github.io/distiller/algo_quantization.html).

The class `QuantCalibrationStatsCollector` collects the statistics for defining the range $r = max - min$.  

Each forward pass, the collector records the values of inputs and outputs, for each layer:
- absolute over all batches min, max (stored in `min`, `max`)
- average over batches, per batch min, max (stored in `avg_min`, `avg_max`)
- mean
- std
- shape of output tensor  

All these values can be used to define the range of quantization, e.g. we can use the absolute `min`, `max` to define the range.

In [9]:
import os
from distiller.data_loggers import QuantCalibrationStatsCollector, collector_context

man_model = torch.load('./manual.checkpoint.pth.tar')
distiller.utils.assign_layer_fq_names(man_model)
collector = QuantCalibrationStatsCollector(man_model)

if not os.path.isfile('manual_lstm_pretrained_stats.yaml'):
    with collector_context(collector) as collector:
        val_loss = evaluate(man_model, val_data)
        collector.save('manual_lstm_pretrained_stats.yaml')

## Prepare the Model For Quantization
  
We quantize the model after the training has completed.  
Here we check the baseline model perplexity, to have an idea how good the quantization is.

In [10]:
from distiller.quantization import PostTrainLinearQuantizer, LinearQuantMode
from copy import deepcopy

# Load and evaluate the baseline model.
man_model = torch.load('./manual.checkpoint.pth.tar')
val_loss = evaluate(man_model, val_data)
print('val_loss:%8.2f\t|\t ppl:%8.2f' % (val_loss, np.exp(val_loss)))

100%|██████████| 622/622 [00:21<00:00, 29.07it/s, val_loss=4.47, ppl=87.7]

val_loss:    4.47	|	 ppl:   87.19


Now we do our magic - __Preparing the model for quantization__.  
The quantizer replaces the layers in out model with their quantized versions.  

In [11]:
# Define the quantizer
quantizer = PostTrainLinearQuantizer(
    deepcopy(man_model),
    model_activation_stats='./manual_lstm_pretrained_stats.yaml')

# Quantizer magic
stats_before_prepare = deepcopy(quantizer.model_activation_stats)
dummy_input = (torch.zeros(1,1).to(dtype=torch.long), man_model.init_hidden(1))
quantizer.prepare_model(dummy_input)

### Net-Aware Quantization

Note that we passes a dummy input to `prepare_model`. This is required for the quantizer to be able to create a graph representation of the model, and to infer the connectivity between the modules.  
Understanding the connectivity of the model is required to enable **"Net-aware quantization"**. This term (coined in [\[1\]](#references), section 3.2.2), means we can achieve better quantization by considering sequences of operations.  
In the case of LSTM, we have an element-wise add operation whose output is split into 4 and fed into either Tanh or Sigmoid activations. Both of these ops saturate at relatively small input values - tanh at approximately $|4|$, and sigmoid saturates at approximately $|6|$. This means we can safely clip the output of the element-wise add operation between $[-6,6]$. `PostTrainLinearQuantizer` detects this patterm and modifies the statistics accordingly.

In [12]:
import pprint
pp = pprint.PrettyPrinter(indent=1)
print('Stats BEFORE prepare_model:')
pp.pprint(stats_before_prepare['rnn.cell_0.eltwiseadd_gate']['output'])

print('\nStats AFTER to prepare_model:')
pp.pprint(quantizer.model_activation_stats['rnn.cell_0.eltwiseadd_gate']['output'])

Stats BEFORE prepare_model:
OrderedDict([('min', -15.612003326416016),
             ('max', 15.450967788696289),
             ('avg_min', -6.393781979404043),
             ('avg_max', 5.522592915806375),
             ('mean', -0.7798242293363614),
             ('std', 1.3385719958875721),
             ('shape', '(10, 6000)')])

Stats AFTER to prepare_model:
OrderedDict([('min', -6.0),
             ('max', 6.0),
             ('avg_min', -6.0),
             ('avg_max', 5.522592915806375),
             ('mean', -0.7798242293363614),
             ('std', 1.3385719958875721),
             ('shape', '(10, 6000)')])


Note the value for `avg_max` did not change, since it was already below the clipping value of $6.0$.

### Inspecting the Quantized Model

Let's see how the model has after being prepared for quantization:

In [13]:
quantizer.model

DistillerRNNModel(
  (encoder): RangeLinearEmbeddingWrapper(
    (wrapped_module): Embedding(33278, 1500)
  )
  (rnn): DistillerLSTM(1500, 1500, num_layers=2, dropout=0.65, bidirectional=False)
  (decoder): RangeLinearQuantParamLayerWrapper(
    mode=SYMMETRIC, num_bits_acts=8, num_bits_params=8, num_bits_accum=32, clip_acts=NONE, per_channel_wts=False, scale_approx_mult_bits=None
    preset_activation_stats=True
    w_scale=107.5208, w_zero_point=0.0000
    in_scale=127.0004, in_zero_point=0.0000
    out_scale=3.6561, out_zero_point=0.0000
    (wrapped_module): Linear(in_features=1500, out_features=33278, bias=True)
  )
)

Note how `encoder` and `decoder` have been replaced with wrapper layers (for the relevant module type), which handle the quantization. The same holds for the internal layers of the `DistillerLSTM` module, which we don't print for brevity sake. To "peek" inside the `DistillerLSTM` module, we need to access it directly. As an example, let's take a look at a couple of the internal layers:

In [14]:
print(quantizer.model.rnn.cell_0.fc_gate_x)
print(quantizer.model.rnn.cell_0.eltwiseadd_gate)

RangeLinearQuantParamLayerWrapper(
  mode=SYMMETRIC, num_bits_acts=8, num_bits_params=8, num_bits_accum=32, clip_acts=NONE, per_channel_wts=False, scale_approx_mult_bits=None
  preset_activation_stats=True
  w_scale=165.1717, w_zero_point=0.0000
  in_scale=126.2964, in_zero_point=0.0000
  out_scale=17.7113, out_zero_point=0.0000
  (wrapped_module): Linear(in_features=1500, out_features=6000, bias=True)
)
RangeLinearQuantEltwiseAddWrapper(
  mode=SYMMETRIC, num_bits_acts=8, num_bits_accum=32, clip_acts=NONE, scale_approx_mult_bits=None
  preset_activation_stats=True
  in_0_scale=17.711265563964844, in_0_zero_point=0.0
  in_1_scale=10.280694961547852, in_1_zero_point=0.0
  out_scale=21.166667938232422, out_zero_point=0.0
  (wrapped_module): EltwiseAdd()
)


## Running the Quantized Model

### Try 1: Initial settings - simple symmetric quantization

Finally, let's go ahead and evaluate the quantized model:

In [15]:
val_loss = evaluate(quantizer.model.to(device), val_data)
print('val_loss:%8.2f\t|\t ppl:%8.2f' % (val_loss, np.exp(val_loss)))

100%|██████████| 622/622 [01:58<00:00,  5.57it/s, val_loss=4.64, ppl=104]  

val_loss:    4.64	|	 ppl:  103.22


As we can see here, the perplexity has increased much - meaning our quantization has damaged the accuracy of our model.

### Try 2: Assymetric, per-channel

Let's try quantizing each channel separately, and making the range of the quantization asymmetric.

In [16]:
quantizer = PostTrainLinearQuantizer(
    deepcopy(man_model),
    model_activation_stats='./manual_lstm_pretrained_stats.yaml',
    mode=LinearQuantMode.ASYMMETRIC_SIGNED,
    per_channel_wts=True
)
quantizer.prepare_model(dummy_input)
quantizer.model

DistillerRNNModel(
  (encoder): RangeLinearEmbeddingWrapper(
    (wrapped_module): Embedding(33278, 1500)
  )
  (rnn): DistillerLSTM(1500, 1500, num_layers=2, dropout=0.65, bidirectional=False)
  (decoder): RangeLinearQuantParamLayerWrapper(
    mode=ASYMMETRIC_SIGNED, num_bits_acts=8, num_bits_params=8, num_bits_accum=32, clip_acts=NONE, per_channel_wts=True, scale_approx_mult_bits=None
    preset_activation_stats=True
    w_scale=PerCh, w_zero_point=PerCh
    in_scale=127.5069, in_zero_point=1.0000
    out_scale=5.0241, out_zero_point=48.0000
    (wrapped_module): Linear(in_features=1500, out_features=33278, bias=True)
  )
)

In [17]:
val_loss = evaluate(quantizer.model.to(device), val_data)
print('val_loss:%8.2f\t|\t ppl:%8.2f' % (val_loss, np.exp(val_loss)))

100%|██████████| 622/622 [02:08<00:00,  5.08it/s, val_loss=4.61, ppl=101]  

val_loss:    4.61	|	 ppl:  100.35


A tiny bit better, but still no good.

### Try 3: Mixed FP16 and INT8

Let us try the half precision (aka FP16) version of the model:

In [18]:
model_fp16 = deepcopy(man_model).half()
val_loss = evaluate(model_fp16, val_data)
print('val_loss: %8.6f\t|\t ppl:%8.2f' % (val_loss, np.exp(val_loss)))

100%|██████████| 622/622 [00:23<00:00, 26.53it/s, val_loss=4.47, ppl=87.7]

val_loss: 4.467975	|	 ppl:   87.18


The result is very close to our original model! That means that the roundoff when quantizing linearly to 8-bit integers is what hurts our accuracy.

Luckily, `PostTrainLinearQuantizer` supports quantizing some/all layers to FP16 using the `fp16` parameter. In light of what we just saw, and as stated in [\[2\]](#References), let's try keeping element-wise operations at FP16, and quantize everything else to 8-bit using the same settings as in try 2.

In [19]:
overrides_yaml = """
.*eltwise.*:
    fp16: true
encoder:
    fp16: true
decoder:
    fp16: true
"""
overrides = distiller.utils.yaml_ordered_load(overrides_yaml)
quantizer = PostTrainLinearQuantizer(
    deepcopy(man_model),
    model_activation_stats='./manual_lstm_pretrained_stats.yaml',
    mode=LinearQuantMode.ASYMMETRIC_SIGNED,
    overrides=overrides,
    per_channel_wts=True
)
quantizer.prepare_model(dummy_input)
quantizer.model

DistillerRNNModel(
  (encoder): FP16Wrapper(
    (wrapped_module): Embedding(33278, 1500)
  )
  (rnn): DistillerLSTM(1500, 1500, num_layers=2, dropout=0.65, bidirectional=False)
  (decoder): FP16Wrapper(
    (wrapped_module): Linear(in_features=1500, out_features=33278, bias=True)
  )
)

In [20]:
val_loss = evaluate(quantizer.model.to(device), val_data)
print('val_loss:%8.6f\t|\t ppl:%8.2f' % (val_loss, np.exp(val_loss)))

100%|██████████| 622/622 [01:04<00:00,  9.68it/s, val_loss=4.47, ppl=87.7] 

val_loss:4.468113	|	 ppl:   87.19


The accuracy is still holding up very well, even though we quantized the inner linear layers!  

### Try 4: Clipping Activations

Now, lets try to choose different boundaries for `min`, `max`.  
Instead of using absolute ones, we take the average of all batches (`avg_min`, `avg_max`), which is an indication of where usually most of the boundaries lie. This is done by specifying the `clip_acts` parameter to `ClipMode.AVG` or `"AVG"` in the quantizer ctor:

In [21]:
overrides_yaml = """
encoder:
    fp16: true
decoder:
    fp16: true
"""
overrides = distiller.utils.yaml_ordered_load(overrides_yaml)
quantizer = PostTrainLinearQuantizer(
    deepcopy(man_model),
    model_activation_stats='./manual_lstm_pretrained_stats.yaml',
    mode=LinearQuantMode.ASYMMETRIC_SIGNED,
    overrides=overrides,
    per_channel_wts=True,
    clip_acts="AVG"
)
quantizer.prepare_model(dummy_input)
val_loss = evaluate(quantizer.model.to(device), val_data)
print('val_loss:%8.6f\t|\t ppl:%8.2f' % (val_loss, np.exp(val_loss)))

100%|██████████| 622/622 [02:09<00:00,  5.22it/s, val_loss=4.5, ppl=90.3]  

val_loss:4.497443	|	 ppl:   89.79


Great! Even though we quantized all of the layers except the embedding and the decoder - we got almost no accuracy penalty. Lets try quantizing them as well:

In [22]:
quantizer = PostTrainLinearQuantizer(
    deepcopy(man_model),
    model_activation_stats='./manual_lstm_pretrained_stats.yaml',
    mode=LinearQuantMode.ASYMMETRIC_SIGNED,
    per_channel_wts=True,
    clip_acts="AVG"
)
quantizer.prepare_model(dummy_input)
val_loss = evaluate(quantizer.model.to(device), val_data)
print('val_loss:%8.6f\t|\t ppl:%8.2f' % (val_loss, np.exp(val_loss)))

100%|██████████| 622/622 [02:12<00:00,  5.10it/s, val_loss=4.51, ppl=90.5] 

val_loss:4.500023	|	 ppl:   90.02


In [23]:
quantizer.model

DistillerRNNModel(
  (encoder): RangeLinearEmbeddingWrapper(
    (wrapped_module): Embedding(33278, 1500)
  )
  (rnn): DistillerLSTM(1500, 1500, num_layers=2, dropout=0.65, bidirectional=False)
  (decoder): RangeLinearQuantParamLayerWrapper(
    mode=ASYMMETRIC_SIGNED, num_bits_acts=8, num_bits_params=8, num_bits_accum=32, clip_acts=AVG, per_channel_wts=True, scale_approx_mult_bits=None
    preset_activation_stats=True
    w_scale=PerCh, w_zero_point=PerCh
    in_scale=129.4670, in_zero_point=1.0000
    out_scale=9.9393, out_zero_point=56.0000
    (wrapped_module): Linear(in_features=1500, out_features=33278, bias=True)
  )
)

Here we see that sometimes quantizing with the right boundaries gives better results than actually using floating point operations (even though they are half precision). 

## Conclusion

Choosing the right boundaries for quantization  was crucial for achieving almost no degradation in accrucay of LSTM.  
  
Here we showed how to use the Distiller quantization API to quantize an RNN model, by converting the PyTorch implementation into a modular one and then quantizing each layer separately.

## References

1. **Jongsoo Park, Maxim Naumov, Protonu Basu, Summer Deng, Aravind Kalaiah, Daya Khudia, James Law, Parth Malani, Andrey Malevich, Satish Nadathur, Juan Miguel Pino, Martin Schatz, Alexander Sidorov, Viswanath Sivakumar, Andrew Tulloch, Xiaodong Wang, Yiming Wu, Hector Yuen, Utku Diril, Dmytro Dzhulgakov, Kim Hazelwood, Bill Jia, Yangqing Jia, Lin Qiao, Vijay Rao, Nadav Rotem, Sungjoo Yoo, Mikhail Smelyanskiy**. Deep Learning Inference in Facebook Data Centers: Characterization, Performance Optimizations and Hardware Implications. [arxiv:1811.09886](https://arxiv.org/abs/1811.09886)

2. **Qinyao He, He Wen, Shuchang Zhou, Yuxin Wu, Cong Yao, Xinyu Zhou, Yuheng Zou**. Effective Quantization Methods for Recurrent Neural Networks. [arxiv:1611.10176](https://arxiv.org/abs/1611.10176)